<a href="https://colab.research.google.com/github/Sai-Lakshmi35/ParallelAlgorithmsCudaProblems/blob/main/Lab5_ParallelAlgorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-9x9zyirh
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-9x9zyirh
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5cd225851b7638f3f6d55a19328295f16c014079
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.0.3-py3-none-any.whl size=7432 sha256=ad99cf1e6175f37f13398802e9e82d42885745e385f9d1c2ce678f3d359f3a95
  Stored in directory: /tmp/pip-ephem-wheel-cache-i5hm742a/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [ ]:
pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter


Source files will be saved in "/tmp/tmpe3xc0fr6".


In [ ]:
%%cuda
#include <stdlib.h>
#include <stdio.h>
#include <time.h>
/* Every thread gets exactly one value in the unsorted array. */
#define THREADS 512 // 2^9
#define BLOCKS 32768 // 2^15
#define NUM_VALS THREADS*BLOCKS
void print_elapsed(clock_t start, clock_t stop)
{
  double elapsed = ((double) (stop - start)) / CLOCKS_PER_SEC;
  printf("Elapsed time: %.3fs\n", elapsed);
}
float random_float()
{
  return (float)rand()/(float)RAND_MAX;
}
void array_print(float *arr, int length)
{
  int i;
  for (i = 0; i < length; ++i) {
    printf("%1.3f ",  arr[i]);
  }
  printf("\n");
}
void array_fill(float *arr, int length)
{
  srand(time(NULL));
  int i;
  for (i = 0; i < length; ++i) {
    arr[i] = random_float();
  }
}
__global__ void bitonic_sort_step(float *dev_values, int j, int k)
{
  unsigned int i, ixj; /* Sorting partners: i and ixj */
  i = threadIdx.x + blockDim.x * blockIdx.x;
  ixj = i^j;
  /* The threads with the lowest ids sort the array. */
  if ((ixj)>i) {
    if ((i&k)==0) {
      /* Sort ascending */
      if (dev_values[i]>dev_values[ixj]) {
        /* exchange(i,ixj); */
        float temp = dev_values[i];
        dev_values[i] = dev_values[ixj];
        dev_values[ixj] = temp;
      }
    }
    if ((i&k)!=0) {
      /* Sort descending */
      if (dev_values[i]<dev_values[ixj]) {
        /* exchange(i,ixj); */
        float temp = dev_values[i];
        dev_values[i] = dev_values[ixj];
        dev_values[ixj] = temp;
      }
    }
  }
}
/**
 * Inplace bitonic sort using CUDA.
 */
void bitonic_sort(float *values)
{
  float *dev_values;
  size_t size = NUM_VALS * sizeof(float);
  cudaMalloc((void**) &dev_values, size);
  cudaMemcpy(dev_values, values, size, cudaMemcpyHostToDevice);
  dim3 blocks(BLOCKS,1);    /* Number of blocks   */
  dim3 threads(THREADS,1);  /* Number of threads  */
  int j, k;
  /* Major step */
  for (k = 2; k <= NUM_VALS; k <<= 1) {
    /* Minor step */
    for (j=k>>1; j>0; j=j>>1) {
      bitonic_sort_step<<<blocks, threads>>>(dev_values, j, k);
    }
  }
  cudaMemcpy(values, dev_values, size, cudaMemcpyDeviceToHost);
  cudaFree(dev_values);
}

int main(void)
{
  clock_t start, stop;
  float *values = (float*) malloc( NUM_VALS * sizeof(float));
  array_fill(values, NUM_VALS);
  start = clock();
  bitonic_sort(values); /* Inplace */
  stop = clock();
  print_elapsed(start, stop);
}

Elapsed time: 0.450s



In [ ]:
%%cuda
#include <stdlib.h>
#include <stdio.h>
#include <time.h>
__global__ void MergeSort(int *nums, int *temp, int n) {
    int tid = threadIdx.x + blockDim.x * blockIdx.x;
    for (int i = 2; i < 2 * n; i *= 2) {
        int len = i;
        if (n - tid < len) len = n - tid;
        if (tid % i == 0) {
            int *seqA = &nums[tid], lenA = i / 2, j = 0;
            int *seqB = &nums[tid + lenA], lenB = len - lenA, k = 0;
            int p = tid;
            while (j < lenA && k < lenB) {
                if (seqA[j] < seqB[k]) {
                    temp[p++] = seqA[j++];
                } else {
                    temp[p++] = seqB[k++];
                }
            }
            while (j < lenA)
                temp[p++] = seqA[j++];
            while (k < lenB)
                temp[p++] = seqB[k++];
            for (int j = tid; j < tid + len; j++)
                nums[j] = temp[j];
        }
        __syncthreads();
    }
}
int main() {
    clock_t start_t = clock();
    int long size = 1024 * 1024;
    int *nums = (int*)malloc(sizeof(int) * size);
    srand(time(0));
    for (int long i = 0; i < size; ++i) {
        nums[i] = rand();
        //printf("%d\t %i", nums[i],i);
       }
    int *dNums;
    cudaMalloc((void**)&dNums, sizeof(int) * size);
    cudaMemcpy(dNums, nums, sizeof(int) * size, cudaMemcpyHostToDevice);
    int *dTemp;
    cudaMalloc((void**)&dTemp, sizeof(int) * size);
    dim3 threadPerBlock(1024);
    dim3 blockNum((size + threadPerBlock.x - 1) / threadPerBlock.x);
    MergeSort<<<blockNum, threadPerBlock>>>(dNums, dTemp, size);
    cudaMemcpy(nums, dNums, sizeof(int) * size, cudaMemcpyDeviceToHost);
    /* for (int long i = 0; i < size; ++i) {
         printf("%d ", nums[i]);
     }*/
     printf("\n");
    free(nums);
    cudaFree(dNums);
    cudaFree(dTemp);
    printf("Number of numbers: %d\n", size);
    printf("Merge Sorting time: %fms\n", clock() - start_t);
}


Number of numbers: 1048576
Merge Sorting time: 0.000000ms

